In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../Cases/Wisconsin/BreastCancer.csv')
df.head()

,Code,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
0,61634,5,4,3,1,2,2,2,3,1,Benign
1,63375,9,1,2,6,4,10,7,7,2,Malignant
2,76389,10,4,7,2,2,8,6,1,1,Malignant
3,95719,6,10,10,10,8,10,7,10,7,Malignant
4,128059,1,1,1,1,2,5,5,1,1,Benign


In [4]:
X, y = df.drop(['Code', 'Class'], axis=1), df['Class']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [13]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier([('dtc', DecisionTreeClassifier(random_state=24)),
                           ('svml', SVC(kernel='linear')),
                           ('svmr', SVC(kernel='rbf'))])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9761904761904762

### Human Resources

In [22]:
hr = pd.read_csv('../Cases/human-resources-analytics/HR_comma_sep.csv')
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [23]:
X, y = hr.drop('left', axis=1), hr['left']
X = pd.get_dummies(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [28]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
ct = make_column_transformer(
    (OneHotEncoder(drop='first'), make_column_selector(dtype_include=object)),
    ('passthrough', make_column_selector(dtype_exclude=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')


In [20]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [21]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier([('dtc', DecisionTreeClassifier(random_state=24)),
                           ('svml', SVC(kernel='linear')),
                           ('svmr', SVC(kernel='rbf'))])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8241831518115137

In [34]:
pipe = Pipeline(
    steps=[
        ('preprocessor', ct),  # Add ColumnTransformer step
        ('voting', voting)      # Add VotingClassifier step
    ]
)

In [35]:
pipe.fit(X_train, y_train)

# Make predictions
y_pred = pipe.predict(X_test)


In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8241831518115137